## Exploratory Data Analysis for Microsoft's New Movie Studio

## Overview

In this project, we will perform exploratory data analysis (EDA) to generate insights for Microsoft, 
helping them decide what type of films to create for their new movie studio. We will use various datasets 
related to movies, analyze trends, and provide actionable recommendations based on our findings.


## Business Problem

Microsoft sees all the big companies creating original video content and they want to get in on the fun. 
They have decided to create a new movie studio but lack knowledge about creating movies. Our task is to explore 
what types of films are currently doing the best at the box office and translate those findings into actionable insights 
for Microsoft's new movie studio.

## Objectives

The objectives of this project are to:
1. Identify the top-performing movie genres at the box office by analyzing historical box office data.
2. Determine the relationship between movie ratings and box office success to recommend the ideal quality standard for new films.
3. Analyze production budgets and their impact on box office returns to suggest optimal budget ranges for new movie projects.

## Loading Data

In [2]:
import pandas as pd 
import sqlite3

In [3]:
bom_movie_gross = pd.read_csv('data/bom.movie_gross.csv')
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv', sep='\t')
rt_reviews = pd.read_csv('data/rt.reviews.tsv', sep='\t', encoding='ISO-8859-1')
tmdb_movies = pd.read_csv('data/tmdb.movies.csv')
tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

conn = sqlite3.connect('data/im.db')
imdb_movie_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
imdb_movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)

In [12]:

print("Movie Gross Data:")
print(bom_movie_gross.head())
print("-------------------------------------------------------------")

print("Movie Info Data:")
print(rt_movie_info.head())
print("-------------------------------------------------------------")

print("Reviews Data:")
print(rt_reviews.head())
print("-------------------------------------------------------------")

print("TMDB Movies Data:")
print(tmdb_movies.head())
print("-------------------------------------------------------------")

print("Movie Budgets Data:")
print(tn_movie_budgets.head())
print("-------------------------------------------------------------")

print("Movie Basics Data:")
print(imdb_movie_basics.head())
print("-------------------------------------------------------------")

print("Movie Ratings Data:")
print(imdb_movie_ratings.head())

Movie Gross Data:
                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  
-------------------------------------------------------------
Movie Info Data:
   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1   3  New York City, not-too-distant-future: Eric Pa...      R   
2   5  Illeana Douglas delivers a superb performance ...      R   
3   6  Michael Douglas runs afoul of a treacherous s

## Data Cleaning